In [ ]:
import re
import unicodedata
import pandas as pd
import nltk
from pydataset import data
import env

# Do your work for this exercise in a file named explore.

# 1. Spam Data
- a. Load the spam data set.
- b. Create and explore bigrams for the spam data. Visualize them with a word cloud. How do they compare with the ham bigrams?
- c. Is there any overlap in the bigrams for the spam data and the ham data?
- d. Create and explore with trigrams (i.e. a n-gram with an n of 3) for both the spam and ham data.




In [ ]:
''' function to connect to CodeUp SQL database'''
def get_connection(db, user=env.username, host=env.host, password=env.password):
    return f'mysql+pymysql://{env.username}:{env.password}@{env.host}/{db}'

In [ ]:
url= get_connection('spam_db')
sql= 'select * from spam'

In [ ]:
df= pd.read_sql(sql,url, index_col='id')
df.head()

In [ ]:
df.label.value_counts()

In [ ]:
# create a data frame with the percentage of ham and spam
labels= pd.concat([df.label.value_counts(),df.label.value_counts(normalize=True)],axis=1)
labels.columns=['n','percent']
labels
                  
                   

In [ ]:
 #lets do some data preparation
    
import unicodedata
import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pandas as pd

In [ ]:
# creating a cleaning function that will lower the text, encode using ascii and decode it back
# this function also replaces anything thats not a-z, 0-9, and white space with nothing
def basic_clean(text):
    text = unicodedata.normalize('NFKD', text.lower())\
        .encode('ascii', 'ignore')\
        .decode('utf-8', 'ignore')
    return re.sub(r"[^a-z0-9\s]", '', text)

In [ ]:
# create a new column that is the cleaned version of the selected column and name that column text_cleaned
df['text_cleaned'] = df.text.apply(basic_clean)
df.head()

In [ ]:
# lets create a tokenize function

def tokenize(string):
    '''
    This function takes in a string and
    returns a tokenized string.
    '''
    # Create tokenizer.
    tokenizer = nltk.tokenize.ToktokTokenizer()
    
    # Use tokenizer
    string = tokenizer.tokenize(string, return_str=True)
    
    return string

In [ ]:
# add a tokenized column to the dataframe by applying tokenize function 
df['text_tokenized']= df.text_cleaned.apply(tokenize)
df.head(2)

In [ ]:
# lets create a lemmatize function

def lemmatize(string):
    '''
    This function takes in string for and
    returns a string with words lemmatized.
    '''
    # Create the lemmatizer.
    wnl = nltk.stem.WordNetLemmatizer()
    
    # Use the lemmatizer on each word in the list of words we created by using split.
    lemmas = [wnl.lemmatize(word) for word in string.split()]
    
    # Join our list of words into a string again and assign to a variable.
    string = ' '.join(lemmas)
    
    return string

In [ ]:
# lets create a new column that is the lemmatized version of the tokenized text and name that column text_lemmatized

df['text_lemmatized'] = df.text_tokenized.apply(lemmatize)
df.head()

In [ ]:
def remove_stopwords(string, extra_words=[], exclude_words=[]):
    '''
    This function takes in a string, optional extra_words and exclude_words parameters
    with default empty lists and returns a string.
    '''
    # Create stopword_list.
    stopword_list = stopwords.words('english')
    
    # Remove 'exclude_words' from stopword_list to keep these in my text.
    stopword_list = set(stopword_list) - set(exclude_words)
    # Add in 'extra_words' to stopword_list.
    stopword_list = stopword_list.union(set(extra_words))
    
    # Split words in string.
    words = string.split()
    
    # Create a list of words from my string with stopwords removed and assign to variable.
    filtered_words = [word for word in words if word not in stopword_list]
    
    # Join words in the list back into strings and assign to a variable.
    string_without_stopwords = ' '.join(filtered_words)
    
    return string_without_stopwords

In [ ]:
# lets filter the text by removing the stop words and give it a name text_filtered

df['text_filtered'] = df.text_lemmatized.apply(remove_stopwords)
df.head()

In [ ]:
# lets split the filtered text and then substitute anything thats not a-z, 0-9, white space or single item
# with nothing 
words = [re.sub(r'([^a-z0-9\s]|\s.\s)', '', doc).split() for doc in df.text_filtered]

# lets concat into the main dataframe
# column name will be words, and the column will contain lists of the words in each doc
df = pd.concat([df, pd.DataFrame({'words': words})], axis=1)

In [ ]:
df

In [ ]:
# to get all of the hamwords, we are filtering the data by label == ham and then joining all of the filtered text

ham_words = ' '.join(df[df.label=='ham'].text_filtered)

# to get all the spamwords, we are filtering the data by label == spam and joining all of the filtered text
spam_words = ' '.join(df[df.label=='spam'].text_filtered)

# doing the same thing but without filtering the dataframe and calling it all words
all_words = ' '.join(df.text_filtered)

# substituting space anything space with nothing in all three of the words
ham_words = re.sub(r'\s.\s', '', ham_words)
spam_words = re.sub(r'\s.\s', '', spam_words)
all_words = re.sub(r'\s.\s', '', all_words)

spam_words

In [ ]:
# lets get the frequency of the words within each word group

ham_freq = pd.Series(ham_words.split()).value_counts()

spam_freq = pd.Series(spam_words.split()).value_counts()

all_freq = pd.Series(all_words.split()).value_counts()

# check the top 5 word counts in all words
all_freq.head()

In [ ]:
# lets create a dataframe that holds the word count from all words, ham words, and spam words

word_counts = (pd.concat([all_freq, ham_freq, spam_freq], axis=1, sort=True)
               .set_axis(['all', 'ham', 'spam'], axis=1, inplace=False)
               .fillna(0)
               .apply(lambda s: s.astype(int))
              )


# lets take a look at the dataframe
word_counts.head()


# lets sort

word_counts.sort_values(by='all', ascending=False).head(10)

**Takeaways:**

- We can see a certain words being used more frequently in one category vs the other.  im is barely used in spam

- **dont** is barely used in spam

- **free** is mostly used in spam

- **call** is more frequent in spam

In [ ]:
# lets check the words that are only used in ham
word_counts[word_counts.spam==0].sort_values(by='ham').tail(10)

In [ ]:
# lets check the words that are only used in spam
word_counts[word_counts.ham==0].sort_values(by='spam').tail(10)

In [ ]:
df.head()

In [ ]:
# lets create a column that holds the count of words in column words
df['doc_length'] = [len(wordlist) for wordlist in df['words']]

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
word_counts.assign(p_spam = word_counts.spam/word_counts['all'],
                  p_ham = word_counts.ham/word_counts['all']).sort_values(by = 'all')[['p_spam', 'p_ham']]\
.tail(20).sort_values('p_ham').plot.barh(stacked = True)
    
plt.title('Proportion of Spam vs Ham for the 20 most commot words in our text sample')

In [ ]:
word_counts.assign(p_spam = word_counts.spam/word_counts['all'],
                  p_ham = word_counts.ham/word_counts['all']).sort_values('all')[['p_spam', 'p_ham']].tail(20).sort_values('p_ham')

In [ ]:
from wordcloud import WordCloud

In [ ]:
ham_cloud = WordCloud(background_color = 'white', height = 1000, width = 1000).generate(ham_words)

plt.imshow(ham_cloud)

In [ ]:
spam_cloud = WordCloud(background_color = 'white', height = 1000, width = 1000).generate(spam_words)
plt.imshow(spam_cloud)

In [ ]:
axs = plt.axes([0.25,1,.5,.5]), plt.axes([0.8,1,.5,.5])

axs[0].imshow(ham_cloud)
axs[1].imshow(spam_cloud)

In [ ]:
# creating a bigram column that holds the bigram of the words in each observation

df['bigrams'] = [list(nltk.ngrams(wordlist, 2)) for wordlist in df.words]
df.head()

In [ ]:
# lets create a bigrams of all of the text in ham and all of the text in spam

ham_bigrams = pd.Series(list(nltk.ngrams(ham_words.split(), 2))).value_counts().head(20)
spam_bigrams = pd.Series(list(nltk.ngrams(spam_words.split(), 2))).value_counts().head(20)
spam_bigrams

In [ ]:
# lets plot these bigrams 

plt.subplot(121)
ham_bigrams.plot.barh(color = 'purple', width = 0.9, figsize = (10,10), alpha = 0.8)
plt.title('20 most frequently occuring ham bigrams')
plt.ylabel('Bigram')
plt.xlabel('Frequency')

plt.subplot(122)
spam_bigrams.plot.barh(color = 'purple', width = 0.9, figsize = (10,10), alpha = 0.8)
plt.title('20 most frequently occuring spam bigrams')
plt.ylabel('Bigram')
plt.xlabel('Frequency')
plt.tight_layout()

In [ ]:
# lets explore tri-grams
ham_trigrams = pd.Series(list(nltk.ngrams(ham_words.split(), 3))).value_counts().head(20)
spam_trigrams = pd.Series(list(nltk.ngrams(spam_words.split(), 3))).value_counts().head(20)
spam_trigrams

In [ ]:
# lets plot these trigrams 

plt.subplot(121)
ham_trigrams.plot.barh(color = 'purple', width = 0.9, figsize = (10,10), alpha = 0.8)
plt.title('20 most frequently occuring ham trigrams')
plt.ylabel('trigram')
plt.xlabel('Frequency')

plt.subplot(122)
spam_trigrams.plot.barh(color = 'purple', width = 0.9, figsize = (10,10), alpha = 0.8)
plt.title('20 most frequently occuring spam trigrams')
plt.ylabel('trigram')
plt.xlabel('Frequency')
plt.tight_layout()

# 2. Explore the blog articles using the techniques discussed in the exploration lesson.

In [5]:
import acquire
import prepare
import re
import json
import pandas as pd

In [7]:
blogs_df= acquire.get_blog_articles()
blogs_df.head()

AttributeError: 'NoneType' object has no attribute 'text'

# 3. Explore the news articles using the techniques discussed in the exploration lesson. Use the category variable when exploring.

In [6]:
news_df= acquire.get_news_articles()
news_df

[{'title': "Don't eff this up: Bezos recalls warning from son for Lord of Rings' prequel",
  'content': 'Ahead of the debut of The Lord of the Rings\' prequel \'Rings of Power\' on Amazon\'s Prime Video, Jeff Bezos revealed a conversation he had with his son. He said, "My son came up to me one day, he looked me in the eyes, very sincerely, and he said: \'Dad, please don\'t eff this up\'...And he was right."',
  'category': 'business'},
 {'title': "Snap CEO confirms 20% job cuts, says 'We must reduce cost to avoid ongoing losses'",
  'content': 'In a letter to staff posted on Snap’s website, CEO Evan Spiegel has confirmed 20% job cuts at the company as the latest quarterly revenue growth of 8% was "well below" expectations. Spiegel wrote, "Unfortunately, given our current lower rate of revenue growth, it has become clear that we must reduce our cost structure to avoid incurring significant ongoing losses."',
  'category': 'business'},
 {'title': 'Price of commercial LPG cylinders cut by

In [8]:
news_df= pd.DataFrame(news_df)
news_df.head()

,title,content,category
0,Don't eff this up: Bezos recalls warning from ...,Ahead of the debut of The Lord of the Rings' p...,business
1,"Snap CEO confirms 20% job cuts, says 'We must ...","In a letter to staff posted on Snap’s website,...",business
2,Price of commercial LPG cylinders cut by up to...,State-owned fuel retailers on Thursday announc...,business
3,Chairman of Russia's 2nd largest oil firm dies...,The chairman of Russia's second-largest oil pr...,business
4,SpiceJet shares fall nearly 15% after CFO resi...,SpiceJet shares declined nearly 15% during Thu...,business


In [9]:
news_df= prepare.prepare_data(news_df,'content')

In [10]:
news_df.head()

,title,content,category,clean,stemmed,lemmatized
0,Don't eff this up: Bezos recalls warning from ...,Ahead of the debut of The Lord of the Rings' p...,business,ahead debut lord rings prequel rings power ama...,ahead debut lord ring prequel ring power amazo...,ahead debut lord ring prequel ring power amazo...
1,"Snap CEO confirms 20% job cuts, says 'We must ...","In a letter to staff posted on Snap’s website,...",business,letter staff posted snaps website ceo evan spi...,letter staff post snap websit ceo evan spiegel...,letter staff posted snap website ceo evan spie...
2,Price of commercial LPG cylinders cut by up to...,State-owned fuel retailers on Thursday announc...,business,stateowned fuel retailers thursday announced r...,stateown fuel retail thursday announc reduct p...,stateowned fuel retailer thursday announced re...
3,Chairman of Russia's 2nd largest oil firm dies...,The chairman of Russia's second-largest oil pr...,business,chairman russias secondlargest oil producer lu...,chairman russia secondlargest oil produc lukoi...,chairman russia secondlargest oil producer luk...
4,SpiceJet shares fall nearly 15% after CFO resi...,SpiceJet shares declined nearly 15% during Thu...,business,spicejet shares declined nearly 15 thursdays i...,spicejet share declin nearli 15 thursday intra...,spicejet share declined nearly 15 thursday int...


In [11]:
# lets remove unnecessary characters from the lemmatized and then split it to get the list of the list of words

words = [re.sub(r'([^a-z0-9\s]|\s.\s)','',doc).split() for doc in news_df.lemmatized]

In [12]:
news_df = pd.concat([news_df, pd.DataFrame({'words':words})], axis = 1)

In [13]:
news_df

,title,content,category,clean,stemmed,lemmatized,words
0,Don't eff this up: Bezos recalls warning from ...,Ahead of the debut of The Lord of the Rings' p...,business,ahead debut lord rings prequel rings power ama...,ahead debut lord ring prequel ring power amazo...,ahead debut lord ring prequel ring power amazo...,"[ahead, debut, lord, ring, prequel, ring, powe..."
1,"Snap CEO confirms 20% job cuts, says 'We must ...","In a letter to staff posted on Snap’s website,...",business,letter staff posted snaps website ceo evan spi...,letter staff post snap websit ceo evan spiegel...,letter staff posted snap website ceo evan spie...,"[letter, staff, posted, snap, website, ceo, ev..."
2,Price of commercial LPG cylinders cut by up to...,State-owned fuel retailers on Thursday announc...,business,stateowned fuel retailers thursday announced r...,stateown fuel retail thursday announc reduct p...,stateowned fuel retailer thursday announced re...,"[stateowned, fuel, retailer, thursday, announc..."
3,Chairman of Russia's 2nd largest oil firm dies...,The chairman of Russia's second-largest oil pr...,business,chairman russias secondlargest oil producer lu...,chairman russia secondlargest oil produc lukoi...,chairman russia secondlargest oil producer luk...,"[chairman, russia, secondlargest, oil, produce..."
4,SpiceJet shares fall nearly 15% after CFO resi...,SpiceJet shares declined nearly 15% during Thu...,business,spicejet shares declined nearly 15 thursdays i...,spicejet share declin nearli 15 thursday intra...,spicejet share declined nearly 15 thursday int...,"[spicejet, share, declined, nearly, 15, thursd..."
...,...,...,...,...,...,...,...
92,"Amitabh Bachchan tests negative for COVID-19, ...",Amitabh Bachchan said he has tested negative f...,entertainment,amitabh bachchan said tested negative covid19 ...,amitabh bachchan said test neg covid19 back wo...,amitabh bachchan said tested negative covid19 ...,"[amitabh, bachchan, said, tested, negative, co..."
93,"'The Kapil...' is my show as well, I'll be bac...",Actor-comedian Krushna Abhishek has said that ...,entertainment,actorcomedian krushna abhishek said kapil shar...,actorcomedian krushna abhishek said kapil shar...,actorcomedian krushna abhishek said kapil shar...,"[actorcomedian, krushna, abhishek, said, kapil..."
94,Wanted my debut to be role which makes differe...,"Actress Sargun Mehta, who will be making her B...",entertainment,actress sargun mehta making bollywood debut cu...,actress sargun mehta make bollywood debut cutt...,actress sargun mehta making bollywood debut cu...,"[actress, sargun, mehta, making, bollywood, de..."
95,Salman Khan's 'Bigg Boss 16' to premiere in Oc...,'Bigg Boss 16' makers have reportedly locked t...,entertainment,bigg boss 16 makers reportedly locked premiere...,bigg boss 16 maker reportedli lock premier dat...,bigg bos 16 maker reportedly locked premiere d...,"[bigg, bos, 16, maker, reportedly, locked, pre..."


In [15]:
# lets join all of the words by category together

business_words = ' '.join(news_df[news_df.title == 'business'].lemmatized)
sports_words = ' '.join(news_df[news_df.title == 'sports'].lemmatized)
technology_words = ' '.join(news_df[news_df.title == 'technology'].lemmatized)
entertainment_words = ' '.join(news_df[news_df.title == 'entertainment'].lemmatized)
all_words = ' '.join(news_df.lemmatized)

In [19]:
# lets split all of the words in each category and convert it to pandas series and get a value count

business_freq = pd.Series(business_words.split()).value_counts()
sports_freq = pd.Series(sports_words.split()).value_counts()
technology_freq = pd.Series(technology_words.split()).value_counts()
entertainment_freq = pd.Series(entertainment_words.split()).value_counts()
all_freq = pd.Series(all_words.split()).value_counts()

/var/folders/h9/174nr99d0zl_7cgtzqpv_gzw144wkq/T/ipykernel_73003/3752690884.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  business_freq = pd.Series(business_words.split()).value_counts()
/var/folders/h9/174nr99d0zl_7cgtzqpv_gzw144wkq/T/ipykernel_73003/3752690884.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sports_freq = pd.Series(sports_words.split()).value_counts()
/var/folders/h9/174nr99d0zl_7cgtzqpv_gzw144wkq/T/ipykernel_73003/3752690884.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  technology_freq = pd.Series(technology_words.split()).value_counts()
/var/folders/h9/174nr99d0zl_7cgtzqpv_gzw144wkq/T/ipykernel_73003/37526

In [20]:
# lets create a dataframe that holds the word counts for each category 

words_count = (pd.concat([all_freq, business_freq, sports_freq, technology_freq, entertainment_freq], axis = 1,sort = True)\
.set_axis(['all', 'business', 'sports', 'technology', 'entertainment'], axis = 1)\
.fillna(0)
.apply(lambda s:s.astype(int))
)

In [21]:
words_count.head()

,all,business,sports,technology,entertainment
05,1,0,0,0,0
1,3,0,0,0,0
10,2,0,0,0,0
100,4,0,0,0,0
1052,1,0,0,0,0


In [23]:
words_count.sort_values('all').head(10)

,all,business,sports,technology,entertainment
05,1,0,0,0,0
meet,1,0,0,0,0
mechthild,1,0,0,0,0
measure,1,0,0,0,0
mathrubootham,1,0,0,0,0
matchmaking,1,0,0,0,0
marriage,1,0,0,0,0
markand,1,0,0,0,0
mehta,1,0,0,0,0
mangalore,1,0,0,0,0
